In [2]:
import pandas as pd
import random
from faker import Faker
import datetime
import hashlib

# Cantidad de registros que deseas generar
num_registros = int(input("¿Cuántos registros deseas generar?: "))

# Lista de comunidades autónomas españolas
comunidades_autonomas = [
    "Andalucía", "Aragón", "Asturias", "Islas Baleares", "Canarias",
    "Cantabria", "Castilla-La Mancha", "Castilla y León", "Cataluña",
    "Extremadura", "Galicia", "Madrid", "Murcia", "Navarra", "La Rioja",
    "Comunidad Valenciana", "País Vasco"
]

# Crear un generador de datos falsos
fake = Faker()

# Crear listas vacías para almacenar los datos
nombres = []
apellidos = []
num_identificacion = []
fecha_nacimiento = []
fecha_jubilacion = []
bases_cotizacion = []  # Agregada la lista de bases de cotización
anos_cotizados = []
comunidad_autonoma = []
invalidez = []  # Nueva columna de tipo booleano

# Rango realista para bases de cotización (por ejemplo, de 1000 a 3000)
base_min = 1000
base_max = 3000

# Años para considerar la edad de jubilación
edad_jubilacion = 65

# Generar datos aleatorios
for _ in range(num_registros):
    nombres.append(fake.first_name())
    apellidos.append(fake.last_name())
    
    # Generar un número de identificación único usando hash
    num_identificacion_hash = hashlib.sha256(f"{fake.ssn()}".encode()).hexdigest()
    num_identificacion.append(num_identificacion_hash)

    # Generar años cotizados
    anos = random.randint(15, 45)  # Mínimo de 15 años cotizados
    meses = random.randint(0, 11)  # Máximo de 11 meses
    
    # Calcular la cotización
    if anos < 25:
        # Calcular años adicionales (25 - años cotizados)
        anos_adicionales = 25 - anos
        cotizacion = 0
        # Definir las bases para los primeros 4 años y el resto hasta 25
        if anos_adicionales <= 4:
            cotizacion = (anos_adicionales * 12 + meses) * 1000 # Cotización simulada de años sin cotizar cuando son menos de 4
            for _ in range(300-(anos_adicionales * 12 + meses)):# Cotización simulada de años sin cotizar más los cotizados
                cotizacion += random.uniform(base_min, base_max)
        else:
            cotizacion = (4 * 12) * 1000 + ((anos_adicionales - 4) * 12) * 500 # Cotización simulada de años sin cotizar cuando son más de 4
            for _ in range(300-(anos_adicionales * 12 + meses)):# Cotización simulada de años sin cotizar más los cotizados
                cotizacion += random.uniform(base_min, base_max)
    else:
        # Años de cotización iguales o superiores a 25
        cotizacion = 0
        for _ in range(300):
            cotizacion += random.uniform(base_min, base_max)
    
    bases_cotizacion.append(round(cotizacion, 2))  # Agregada la cotización con 2 decimales
    
    anos_cotizados.append(f"{anos} años {meses} meses")
    
    # Calcular la edad y ajustar la fecha de nacimiento
    if anos < 37 or (anos == 37 and meses < 9):
        edad_minima = 66  # Edad mínima para aquellos con menos de 37 años y 9 meses cotizados
        edad_maxima = 75
    else:
        edad_minima = 65  # Edad mínima para aquellos con 37 años o más cotizados
        edad_maxima = 65
    fecha_nacimiento.append(fake.date_of_birth(tzinfo=None, minimum_age=edad_minima, maximum_age=edad_maxima))

    # Generar la fecha de jubilación (siempre en 2023)
    fecha_jubilacion.append(datetime.date(2023, random.randint(1, 12), random.randint(1, 28)))

    # Seleccionar una comunidad autónoma al azar
    comunidad_autonoma.append(random.choice(comunidades_autonomas))

    # Generar un valor de invalidez aleatorio (booleano)
    invalidez.append(random.choice([True, False]))

# Crear un DataFrame con los datos
data = pd.DataFrame({
    'Nombre': nombres,
    'Apellidos': apellidos,
    'Numero_Identificacion': num_identificacion,
    'Fecha de Nacimiento': fecha_nacimiento,
    'Fecha de Jubilación': fecha_jubilacion,
    'Bases de Cotización': bases_cotizacion,
    'Años Cotizados': anos_cotizados,
    'Comunidad Autónoma': comunidad_autonoma,
    'Invalidez': invalidez  
})

# Guardar los datos en un archivo CSV
data.to_csv('datos_jubilacion.csv', index=False, encoding='utf-8')

print(f'Se han generado {num_registros} registros y se han guardado en "datos_jubilacion.csv."')

# Crear la tabla de rentas jubilados con los mismos valores de identificación y renta aleatoria
num_identificacion_nueva = num_identificacion  # Copiar los valores de identificación

rentas_anteriores = []
for _ in range(num_registros):
    renta_anterior = round(random.uniform(0, 12000), 2)
    rentas_anteriores.append(renta_anterior)

rentas_jubilados = pd.DataFrame({
    'Numero_Identificacion': num_identificacion_nueva,
    'Renta Año Anterior': rentas_anteriores
})

# Guardar la tabla de rentas jubilados en un archivo CSV
rentas_jubilados.to_csv('rentas_jubilados.csv', index=False, encoding='utf-8')

print(f'Se han generado la tabla "rentas_jubilados.csv" con {num_registros} registros.')


Se han generado 450 registros y se han guardado en "datos_jubilacion.csv."
Se han generado la tabla "rentas_jubilados.csv" con 450 registros.


In [3]:
import pandas as pd

# Función para calcular la Base Reguladora
def calcular_base_reguladora(row):
    bases_cotizacion = row['Bases de Cotización']
    anos_cotizados = int(row['Años Cotizados'].split()[0])
    meses_cotizados = int(row['Años Cotizados'].split()[2])

    total_meses_cotizados = (anos_cotizados - 15) * 12 + meses_cotizados
    base_reguladora = bases_cotizacion / 350
    return round(base_reguladora, 2), total_meses_cotizados

# Función para calcular la Pensión
def calcular_pension(row):
    base_reguladora, total_meses_cotizados = row['Base Reguladora'], row['Total Meses Cotizados']
    anos_cotizados = int(row['Años Cotizados'].split()[0])  # Obtener el total de años cotizados
    
    pension = 0
    
    if anos_cotizados >= 36:
        pension = round(base_reguladora, 2)
    else:
        if total_meses_cotizados <= 106:
            pension = base_reguladora/2 + (0.0021 * total_meses_cotizados * base_reguladora)
        else:
            base_reguladora1 = round(0.0021 * 106 * base_reguladora, 2)
            base_reguladora2 = round(0.0019 * (total_meses_cotizados - 106) * base_reguladora, 2)
            pension = base_reguladora / 2 + base_reguladora1 + base_reguladora2

    return round(pension, 2)

# Leer el archivo CSV
file_path = 'datos_jubilacion.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Calcular la Base Reguladora y Total Meses Cotizados
df[['Base Reguladora', 'Total Meses Cotizados']] = df.apply(calcular_base_reguladora, axis=1, result_type='expand')

# Calcular la Pensión y agregarla al DataFrame
df['Pensión'] = df.apply(calcular_pension, axis=1)

# Agregar la columna "Pensión Final" con los 300 euros si hay invalidez
df['Pensión Final'] = df.apply(lambda row: row['Pensión'] + 300 if row['Invalidez'] else row['Pensión'], axis=1)

# Guardar los datos en un nuevo archivo CSV con formato de números y codificación utf-8
output_file = 'datos_jubilacion_calculados.csv'
df.to_csv(output_file, index=False, float_format='%.2f', encoding='utf-8')

print(f"Los datos procesados se han guardado en {output_file}")


Los datos procesados se han guardado en datos_jubilacion_calculados.csv
